Cosmo MCMC is landing on biased HOD + Cosmology. I'm gonna first look at the code to make preds, and compare to the emulator. 

In [1]:
from pearce.emulator import OriginalRecipe, ExtraCrispy, SpicyBuffalo
from pearce.mocks import cat_dict
import numpy as np
from os import path

In [2]:
import matplotlib
#matplotlib.use('Agg')
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

In [3]:
training_file = '/scratch/users/swmclau2/xi_zheng07_cosmo_lowmsat/PearceRedMagicXiCosmoFixedNd.hdf5'
test_file = '/scratch/users/swmclau2/xi_zheng07_cosmo_test_lowmsat/PearceRedMagicXiCosmoFixedNd_Test.hdf5'

#test_file = '/u/ki/swmclau2/des/xi_cosmo_tester/PearceRedMagicXiCosmoFixedNd_test.hdf5'
em_method = 'gp'
split_method = 'random'

In [4]:
a = 1.0
z = 1.0/a - 1.0

In [5]:
fixed_params = {'z':z}#, 'cosmo': 3}#, 'r':0.53882047}

In [6]:
np.random.seed(0)
emu = SpicyBuffalo(test_file, method = em_method, fixed_params=fixed_params,
                 custom_mean_function = None, downsample_factor = 0.1)

no nans


IOError: [Errno 2] No such file or directory: 'default_metrics.pkl'

In [ ]:
emu.get_param_names()

In [ ]:
v = np.ones_like(emu._emulators[0].get_parameter_vector())*12.0

In [ ]:
if hasattr(emu, "_emulator"):
    emu._emulator.set_parameter_vector(v)
    emu._emulator.recompute()
else:
    for _emulator in emu._emulators:
        _emulator.set_parameter_vector(v)
        _emulator.recompute()

In [ ]:
params = {}
for pname in emu.get_param_names():
    if pname == 'r':
        continue
    low, high = emu.get_param_bounds(pname)
    params[pname] = np.random.uniform(low, high)
    
print params

In [ ]:
fixed_params = {}#'f_c':1.0}#,'logM1': 13.8 }# 'z':0.0}

cosmo_params = {'simname':'testbox', 'boxno': 3, 'realization': 0, 'scale_factors':[1.0], 'system': 'sherlock'}
cat = cat_dict[cosmo_params['simname']](**cosmo_params)#construct the specified catalog!

In [ ]:
cat.load(1.0, HOD='zheng07')

In [ ]:
print len(cat.halocat.halo_table)

In [ ]:
cat._get_cosmo_param_names_vals()[1]

In [ ]:
test_point_idx = 1
test_point_dict = dict(zip(emu.get_param_names(), emu.x[0][test_point_idx]*emu._x_std[0]+emu._x_mean[0]))

In [ ]:
#hod_param_names = ['logM0', 'sigma_logM', 'logM1', 'alpha']
emulation_point = [('logM0', 14.0), ('sigma_logM', 0.2),
                    ('alpha', 1.083),('logM1', 13.7)]#, ('logMmin', 12.233)]

#em_params = {key:test_point_dict[key] for key in hod_param_names}
#em_params = dict(zip(hod_param_names, x_point))
em_params = dict(emulation_point)
em_params.update(fixed_params)

In [ ]:
from scipy.optimize import minimize_scalar
def add_logMmin(hod_params, cat, nd = 1e-4):
    """
    In the fixed number density case, find the logMmin value that will match the nd given hod_params
    :param: hod_params:
        The other parameters besides logMmin
    :param cat:
        the catalog in question
    :return:
        None. hod_params will have logMmin added to it.
    """
    hod_params['logMmin'] = 13.0 #initial guess
    #cat.populate(hod_params) #may be overkill, but will ensure params are written everywhere
    def func(logMmin, hod_params):
        hod_params.update({'logMmin':logMmin})
        calc_nd = cat.calc_analytic_nd(hod_params)
        #print logMmin, calc_nd
        return (calc_nd - nd)**2

    res = minimize_scalar(func, bounds = (12, 16), args = (hod_params,), options = {'maxiter':100}, method = 'Bounded' )
    # assuming this doens't fail
    hod_params['logMmin'] = res.x


In [ ]:
add_logMmin(em_params, cat)

In [ ]:
r_bins = np.logspace(-1.1, 1.6, 19)
rpoints = (r_bins[1:]+r_bins[:-1])/2.0

In [ ]:
em_params['logMmin']

In [ ]:
_xi_vals = []
for i in xrange(5):
    cat.populate(em_params, min_ptcl=100)
    _xi_vals.append(cat.calc_xi(r_bins))

In [ ]:
xi_vals = np.log10(np.array(_xi_vals))
y_calc = xi_vals.mean(axis = 0) #take one example as our xi. could also use the mean, but lets not cheat.
cov = np.cov(xi_vals.T)#/np.sqrt(50)

In [ ]:
print em_params

In [ ]:
y_calc

In [ ]:
y = np.array([_y[test_point_idx] for _y in emu.y])
print y
y+= emu.mean_function([_x[test_point_idx]] for _x in emu.x)#[:,0]
print y

In [ ]:
plt.plot(rpoints, xi_vals.T, alpha = 0.4)
plt.plot(rpoints, y_calc)
plt.plot(rpoints, y )

#plt.plot(rpoints, y_point-0.2)

plt.xscale('log')

In [ ]:
# get cosmo params
del em_params['logMmin']
cpv = cat._get_cosmo_param_names_vals()

cosmo_param_dict = {key: val for key, val in zip(cpv[0], cpv[1])}

#em_params.update( cosmo_param_dict)


In [ ]:
fixed_params = {'z':z, 'cosmo': 3}#, 'r':0.53882047}
train_x, train_y, _, _ = emu.get_data(test_file, fixed_params, None, skip_nans = False)

In [ ]:
train_x

In [ ]:
for idx in xrange(10):
    x_point = train_x[idx*emu.n_bins, :-1]
    y_point = train_y[idx*emu.n_bins:(idx+1)*emu.n_bins]
    pop_params = dict(zip(emu.get_param_names(), x_point))
    
    y_pred = emu.emulate_wrt_r(pop_params)
    
    add_logMmin(pop_params, cat)
    print pop_params

    _xi_vals = []
    for i in xrange(10):
        cat.populate(pop_params, min_ptcl=100)
        _xi_vals.append(cat.calc_xi(r_bins))

    xi_vals = np.log10(np.array(_xi_vals))
    y_calc = xi_vals.mean(axis = 0)
    
    fig = plt.figure(figsize = (14, 6))

    plt.subplot(121)
    plt.plot(rpoints, y_calc, label = 'Sim')
    plt.plot(rpoints, y_point, label = 'Data')
    plt.plot(rpoints, y_pred, label = 'Emu')
    plt.xscale('log')
    plt.legend(loc = 'best')
    plt.subplot(122)
    plt.plot(rpoints, y_calc/y_point, 'Sim')
    plt.plot(rpoints, y_pred/y_point, 'Emu')

    plt.xscale('log')
    plt.show()